<a href="https://colab.research.google.com/github/KodumuruRaja/Feature-Selection/blob/main/Feature-Selection-(Linear%20and%20Logistic-Regression%20with%20L1%20and%20L2%20(%20Lasso%20and%20Ridge)%20Regularization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [ ]:
titanic=sns.load_dataset('titanic')

In [ ]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [ ]:
titanic.drop(labels=['age','deck'],axis=1,inplace=True)

In [ ]:
titanic=titanic.dropna()

In [ ]:
titanic.isnull().sum()

survived       0
pclass         0
sex            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [ ]:
data=titanic[['pclass','sex','sibsp','parch','embarked','who','alone']].copy()

In [ ]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,male,1,0,S,man,False
1,1,female,1,0,C,woman,False
2,3,female,0,0,S,woman,True
3,1,female,1,0,S,woman,False
4,3,male,0,0,S,man,True


In [ ]:
data.isnull().sum()

pclass      0
sex         0
sibsp       0
parch       0
embarked    0
who         0
alone       0
dtype: int64

In [ ]:
sex={'male':0,'female':1}
data['sex']=data['sex'].map(sex)

In [ ]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,S,man,False
1,1,1,1,0,C,woman,False
2,3,1,0,0,S,woman,True
3,1,1,1,0,S,woman,False
4,3,0,0,0,S,man,True


In [ ]:
ports={'S':0,'C':1,'Q':2}
data['embarked']=data['embarked'].map(ports)

In [ ]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,0,man,False
1,1,1,1,0,1,woman,False
2,3,1,0,0,0,woman,True
3,1,1,1,0,0,woman,False
4,3,0,0,0,0,man,True


In [ ]:
who={'man':0,'woman':1,'child':2}
data['who']=data['who'].map(who)

In [ ]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,0,0,False
1,1,1,1,0,1,1,False
2,3,1,0,0,0,1,True
3,1,1,1,0,0,1,False
4,3,0,0,0,0,0,True


In [ ]:
alone={True:1,False:0}
data['alone']=data['alone'].map(alone)

In [ ]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,0,0,0
1,1,1,1,0,1,1,0
2,3,1,0,0,0,1,1
3,1,1,1,0,0,1,0
4,3,0,0,0,0,0,1


In [ ]:
x=data.copy()
y=titanic['survived']

In [ ]:
x.shape,y.shape

((889, 7), (889,))

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=43)

# **Estimation of Coefficients of Linear Regression**

In [ ]:
sel=SelectFromModel(LinearRegression())

In [ ]:
sel.fit(x_train,y_train)

SelectFromModel(estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                           n_jobs=None, normalize=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [ ]:
sel.get_support()

array([ True,  True, False, False, False,  True, False])

In [ ]:
sel.estimator_.coef_

array([-0.13750402,  0.26606466, -0.07470416, -0.0668525 ,  0.04793674,
        0.23857799, -0.12929595])

In [ ]:
mean=np.mean(np.abs(sel.estimator_.coef_))

In [ ]:
mean

0.13727657291370773

In [ ]:
np.abs(sel.estimator_.coef_)

array([0.13750402, 0.26606466, 0.07470416, 0.0668525 , 0.04793674,
       0.23857799, 0.12929595])

In [ ]:
features=x_train.columns[sel.get_support()]

In [ ]:
features

Index(['pclass', 'sex', 'who'], dtype='object')

In [ ]:
x_train_reg=sel.transform(x_train)
x_test_reg=sel.transform(x_test)

In [ ]:
x_train_reg.shape,x_test_reg.shape

((622, 3), (267, 3))

In [ ]:
def run(x_train,x_test,y_train,y_test):
  clf=RandomForestClassifier(n_estimators=100,random_state=0,n_jobs=-1)
  clf=clf.fit(x_train,y_train)
  y_pred=clf.predict(x_test)
  print("Accuracy",accuracy_score(y_test,y_pred))

In [ ]:
%%time
run(x_train_reg,x_test_reg,y_train,y_test)

Accuracy 0.8239700374531835
CPU times: user 244 ms, sys: 37.4 ms, total: 281 ms
Wall time: 352 ms


In [ ]:
%%time
run(x_train,x_test,y_train,y_test)

Accuracy 0.8239700374531835
CPU times: user 272 ms, sys: 43.3 ms, total: 315 ms
Wall time: 359 ms


In [ ]:
x_train.shape,x_test.shape

((622, 7), (267, 7))

# **Logistic Regression Coefficient with L1 Regularization**

In [ ]:
sel=SelectFromModel(LogisticRegression(penalty='l1',C=0.05,solver='liblinear'))
sel.fit(x_train,y_train)
sel.get_support()

array([ True,  True,  True, False, False,  True, False])

In [ ]:
sel.estimator_.coef_

array([[-0.54046104,  0.78061887, -0.14077582,  0.        ,  0.        ,
         0.94088249,  0.        ]])

In [ ]:
x_train_l1=sel.transform(x_train)
x_test_l1=sel.transform(x_test)

In [ ]:
%%time
run(x_train_l1,x_test_l1,y_train,y_test)

Accuracy 0.8277153558052435
CPU times: user 247 ms, sys: 32.2 ms, total: 279 ms
Wall time: 348 ms


# **L2 Regularization**

In [ ]:
sel=SelectFromModel(LogisticRegression(penalty='l2',C=0.05,solver='liblinear'))
sel.fit(x_train,y_train)
sel.get_support()

array([ True,  True, False, False, False,  True, False])

In [ ]:
sel.estimator_.coef_

array([[-0.55749685,  0.85692344, -0.30436065, -0.11841967,  0.2435823 ,
         1.00124155, -0.29875898]])

In [ ]:
x_train_l1=sel.transform(x_train)
x_test_l1=sel.transform(x_test)

In [ ]:
%%time
run(x_train_l1,x_test_l1,y_train,y_test)

Accuracy 0.8239700374531835
CPU times: user 259 ms, sys: 57.4 ms, total: 317 ms
Wall time: 347 ms
